<a href="https://colab.research.google.com/github/griffinfulton-uark/Classification-Model-Project-/blob/main/VectorizeTweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Playing with how we wanted to vectorize tweets, this was for the purpose of sentiment analysis 

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install snscrape
!pip install transformers

     |████████████████████████████████| 4.0 MB 30.3 MB/s 
     |████████████████████████████████| 77 kB 6.9 MB/s 
     |████████████████████████████████| 6.6 MB 54.1 MB/s 
     |████████████████████████████████| 895 kB 46.2 MB/s 
     |████████████████████████████████| 596 kB 58.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import pandas as pd
import snscrape.modules.twitter as sntwitter
from transformers import AutoTokenizer,AutoModelForSequenceClassification
from scipy.special import softmax

In [ ]:
# load model and tokenizer
roberta = "cardiffnlp/twitter-roberta-base-sentiment"

model = AutoModelForSequenceClassification.from_pretrained(roberta)
tokenizer = AutoTokenizer.from_pretrained(roberta)

def sentiment(tweet):
    tweet_words = []
    tweet = tweet
    for word in tweet.split(' '):
        if word.startswith('@') and len(word) > 1:
            word = '@user'

        elif word.startswith('http'):
            word = "http"
        tweet_words.append(word)

    tweet_proc = " ".join(tweet_words)

    labels = ['Negative', 'Neutral', 'Positive']

    # sentiment analysis
    encoded_tweet = tokenizer(tweet_proc, return_tensors='pt')
    output = model(encoded_tweet['input_ids'], encoded_tweet['attention_mask'])
    output = model(**encoded_tweet)

    scores = output[0][0].detach().numpy()
    scores = softmax(scores)

    for i in range(len(scores)):
        l = labels[i]
        s = scores[i]
        return(l,s)

Downloading:   0%|          | 0.00/747 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/476M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [ ]:
## create a data frame from for the tweets from different senatores
sen_tweets = pd.read_csv('/content/drive/MyDrive/EconomicAnalyticsFinalProject/tweet_collection_v4.csv')
party_tweets = pd.read_csv('/content/drive/MyDrive/EconomicAnalyticsFinalProject/platform_tweets.csv')



In [ ]:
### need to change the label of the tweets for this to match the sen tweets data set
#party_tweets.head()
#party_tweets.loc[party_tweets['handle'] == ['GOP', 'R'], 'handle'] = 'R'
party_tweets['User'].value_counts()

TheDemocrats     3562
harrisonjaime    3190
GOP              3085
GOPChairwoman    3081
Name: User, dtype: int64

In [ ]:
import numpy as np
#function to return the respective parties
party_tweets['par'] = " "
party_tweets.loc[ party_tweets['User'] == 'TheDemocrats', 'par'] = 'D'
party_tweets.loc[ party_tweets['User'] == 'harrisonjaime', 'par'] = 'D'
party_tweets.loc[ party_tweets['User'] == 'GOP', 'par'] = 'R'
party_tweets.loc[ party_tweets['User'] == 'GOPChairwoman', 'par'] = 'R'
party_tweets.drop(columns=['handle', 'Party'])

,Unnamed: 0,Date,User,Tweets,par
0,0,2022-04-19 23:39:00+00:00,GOP,March’s encounters passed the previous Biden p...,R
1,1,2022-04-19 22:35:16+00:00,GOP,@JohnJamesMI @GOPChairwoman Americans are tire...,R
2,2,2022-04-19 21:22:00+00:00,GOP,.@GOPChairwoman and @JohnJamesMI know how the ...,R
3,3,2022-04-19 20:46:13+00:00,GOP,@JuliaFriedland When ours are more affordable ...,R
4,4,2022-04-19 19:53:01+00:00,GOP,🚨The rise of Black conservatives is breaking t...,R
...,...,...,...,...,...
12913,3185,2021-01-03 13:31:58+00:00,harrisonjaime,"In the end, folks like @MittRomney will have t...",D
12914,3186,2021-01-03 13:31:58+00:00,harrisonjaime,We should all be disgusted by the treachery of...,D
12915,3187,2021-01-03 13:31:58+00:00,harrisonjaime,"Lindsey was right, we may be witnessing the de...",D
12916,3188,2021-01-03 01:50:42+00:00,harrisonjaime,A must read thread from my friend @SteveSchmid...,D
